In [1]:
pip install pyLDAvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 1.4 MB/s eta 0:00:00a 0:00:010m
Note: you may need to restart the kernel to use updated packages.


In [14]:
import pandas as pd #数据表

In [16]:
import numpy as np #数组

In [18]:
from gensim import corpora, models

In [20]:
import re #正则表达式

In [22]:
import matplotlib.pyplot as plt #画图

In [23]:
import jieba #中文分词

In [26]:
import pyLDAvis #交互化LDA可视化

In [28]:
import pyLDAvis.gensim_models as gensimvis

In [30]:
df = pd.read_excel('text_analysis_weibo_sample.xlsx', index_col=0)

In [32]:
df.head()

,index,标题/微博内容,点赞,转发,评论,账号昵称UID加密,粉丝数,关注数,地域
0,34121,国债：地产行业重磅利好提振风险偏好，期债低开低走 国债期货全线收跌，10年期主力...,0,0,0,e5df796860e68f403bcf9651bab4d42e,0,0,其他
1,40230,#喜迎二十大 忠诚保平安#,0,0,0,6e35cb69ad52f20de5e28197b2e85306,405444,252,广西
2,7714,注意！事关明日教资考试！福建省教育考试院发布补充公告 福建省2022年下半年全国中小学教师...,0,0,0,e6953217442e6c06a7af23eee5e185f2,53264,2177,福建
3,27378,近日，“千年大计”雄安新区迎来五周岁生日。从“一张白纸...,0,0,0,NaN,0,0,北京
4,15435,樊振东牛逼！,0,0,0,344af41eac516375c04dee6325e763cc,8,51,山东


语料预处理

In [35]:
#加载中文停用词词典 可个性化设置

In [193]:
def clean_text(text):
    #剔除数字与符号 仅保留汉字
    processed=re.sub('[^\u4e00-\u9fa5]+',' ', text)
    #分词
    words=jieba.lcut(processed)
    #剔除停用词
    stopwords=['的','了','在','和','为','也','是','我','有','就','较','这','你','我们','等','吧','们','或','都','月','年','去','不','啊','呜呜']
    words=[w for w in words if w not in stopwords]
    return ' '.join(words)

In [195]:
max_length = 100
text = df['标题/微博内容'][0]

if len(text) > max_length:
    print(text[:max_length - 3] + '...')  # 预留3个位置给省略号
else:
    print(text)

  国债：地产行业重磅利好提振风险偏好，期债低开低走     国债期货全线收跌，10年期主力合约跌0.37%，5年期主力合约跌0.24%，2年期主力合约跌0.11%，三大主力合约均创逾一年收盘新...


In [197]:
max_length = 100
text = clean_text(df['标题/微博内容'][0])

if len(text) > max_length:
    print(text[:max_length - 3] + '...')  # 预留3个位置给省略号
else:
    print(text)

  国债   地产 行业 重磅 利好 提振 风险 偏好   期债 低开 低 走   国债 期货 全线 收跌   年期 主力 合约 跌   年期 主力 合约 跌   年期 主力 合约 跌   三大...


In [199]:
df['标题/微博内容']=df['标题/微博内容'].astype(str)
df['微博内容分词']=df['标题/微博内容'].astype(str).apply(clean_text)
df['微博内容分词']=df['微博内容分词'].apply(lambda x: x.split())

In [200]:
df

,index,标题/微博内容,点赞,转发,评论,账号昵称UID加密,粉丝数,关注数,地域,微博内容分词
0,34121,国债：地产行业重磅利好提振风险偏好，期债低开低走 国债期货全线收跌，10年期主力...,0,0,0,e5df796860e68f403bcf9651bab4d42e,0,0,其他,"[国债, 地产, 行业, 重磅, 利好, 提振, 风险, 偏好, 期债, 低开, 低, 走,..."
1,40230,#喜迎二十大 忠诚保平安#,0,0,0,6e35cb69ad52f20de5e28197b2e85306,405444,252,广西,"[喜迎, 二十大, 忠诚, 保平安]"
2,7714,注意！事关明日教资考试！福建省教育考试院发布补充公告 福建省2022年下半年全国中小学教师...,0,0,0,e6953217442e6c06a7af23eee5e185f2,53264,2177,福建,"[注意, 事关, 明日, 教资, 考试, 福建省, 教育, 考试院, 发布, 补充, 公告,..."
3,27378,近日，“千年大计”雄安新区迎来五周岁生日。从“一张白纸...,0,0,0,NaN,0,0,北京,"[近日, 千年, 大计, 雄安, 新区, 迎来, 五周岁, 生日, 从, 一张白纸, 到, ..."
4,15435,樊振东牛逼！,0,0,0,344af41eac516375c04dee6325e763cc,8,51,山东,"[樊振东, 牛, 逼]"
...,...,...,...,...,...,...,...,...,...,...
95,14034,老公好漂亮[舔屏],0,0,0,aa7af97ae98cbe8983e559b948ecfabf,825,191,北京,"[老公, 好, 漂亮, 舔, 屏]"
96,35249,恭喜@张小娜呀娜 1名用户获得【小飞T】。C官方唯一抽奖工具@C抽奖平台 对本次抽奖进行监督...,0,0,2,f538513e5801c275cbcf285517a8ee62,51709,20,北京,"[恭喜, 张小娜, 呀, 娜, 名, 用户, 获得, 小飞, 官方, 唯一, 抽奖, 工具,..."
97,39598,接不动了，居民部门负债率提升空间太有限了。现在动辄几万一平米，总价高，月供高，预期转差，还怎...,0,0,0,2e76c39b6665a0b6f5bdc4c35f36f573,0,59,四川,"[接不动, 居民, 部门, 负债率, 提升, 空间, 太, 有限, 现在, 动辄, 几万, ..."
98,48977,目前电子烟政策已从制定阶段逐渐走向实施阶段，而主要的电子烟公司股价下跌幅度高达70%-...,0,0,0,NaN,0,0,北京,"[目前, 电子, 烟, 政策, 已, 从, 制定, 阶段, 逐渐, 走向, 实施, 阶段, ..."


LDA

In [204]:
texts=[['human','interface','computer']]

In [206]:
dictionary=corpora.Dictionary(texts) #初始化字典

In [208]:
print(dictionary.token2id)

{'computer': 0, 'human': 1, 'interface': 2}


In [210]:
print(dictionary.doc2bow(["human","interface","computer"]))

[(0, 1), (1, 1), (2, 1)]


In [212]:
dictionary.add_documents([["cat","say","meow"],["dog"]]) #add more document(拓展 vocabulary)

In [214]:
print(dictionary.token2id)
print(dictionary.doc2bow(["dog","computer","non_existent_word"]))

{'computer': 0, 'human': 1, 'interface': 2, 'cat': 3, 'meow': 4, 'say': 5, 'dog': 6}
[(0, 1), (6, 1)]


In [216]:
corpus=dictionary.doc2bow(['human','interface','computer'])#根据分词结果创建语料库

In [218]:
corpus

[(0, 1), (1, 1), (2, 1)]

In [220]:
dictionary=corpora.Dictionary(df['微博内容分词'])#根据分词结果创建字典
corpus=[dictionary.doc2bow(text)for text in df['微博内容分词']]#根据分词结果创建语料库

In [222]:
corpus[:10]

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 2),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 2),
  (12, 1),
  (13, 3),
  (14, 3),
  (15, 1),
  (16, 2),
  (17, 2),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 2),
  (23, 1),
  (24, 3),
  (25, 1),
  (26, 2),
  (27, 2),
  (28, 1),
  (29, 1),
  (30, 2),
  (31, 2),
  (32, 1),
  (33, 1),
  (34, 3),
  (35, 2),
  (36, 6),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 2),
  (41, 2),
  (42, 1),
  (43, 2),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 8),
  (50, 1),
  (51, 5),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 2),
  (57, 1),
  (58, 1),
  (59, 2),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 4),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 2),
  (69, 3),
  (70, 3),
  (71, 1),
  (72, 3),
  (73, 9),
  (74, 1),
  (75, 7),
  (76, 1),
  (77, 1),
  (78, 1),
  (79, 1),
  (80, 1),
  (81, 1),
  (82, 1),
  (83, 3),
  (84, 1),
  (85, 1),
  (86, 4),
  (87, 2),
  (88, 1),
  (89, 1),
  (90, 1),
  (91, 1)

In [233]:
#训练LDA模型
lda_model=models.LdaModel(corpus,num_topics=5,id2word=dictionary,passes=15)

In [234]:
#查看主题
topics=lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.020*"电子" + 0.019*"烟" + 0.013*"中国" + 0.011*"考生" + 0.006*"考试"')
(1, '0.007*"数字" + 0.006*"新" + 0.005*"建设" + 0.005*"发展" + 0.005*"报告"')
(2, '0.006*"玉米" + 0.006*"公司" + 0.005*"产品" + 0.005*"主要" + 0.005*"重庆"')
(3, '0.007*"加息" + 0.006*"通胀" + 0.006*"欧洲央行" + 0.005*"英国" + 0.005*"至"')
(4, '0.013*"万吨" + 0.012*"吨" + 0.011*"元" + 0.009*"市场" + 0.008*"库存"')


In [237]:
for index, score in sorted(lda_model[corpus[0]],key=lambda tup:-1*tup[1]):
    print("\nScore:{}\t \nTopic:{}".format(score,lda_model.print_topic(index,10)))


Score:0.938465416431427	 
Topic:0.007*"数字" + 0.006*"新" + 0.005*"建设" + 0.005*"发展" + 0.005*"报告" + 0.005*"期货" + 0.004*"经济" + 0.004*"亿元" + 0.004*"从" + 0.004*"到"

Score:0.061051324009895325	 
Topic:0.013*"万吨" + 0.012*"吨" + 0.011*"元" + 0.009*"市场" + 0.008*"库存" + 0.007*"日" + 0.006*"需求" + 0.006*"预期" + 0.006*"下降" + 0.005*"环比"


In [239]:
def infer_topics(lda_model, document):
    bow=dictionary.doc2bow(document)
    topics=lda_model.get_document_topics(bow)
    return topics

In [245]:
#print topics for each document
documents=df["微博内容分词"].values.tolist()

In [247]:
for i, doc in enumerate(documents[:10]):
    print(doc)
    doc_topics=infer_topics(lda_model,doc)
    print(f"docunent {i+1}:")
    print(doc_topics)
    print()

['国债', '地产', '行业', '重磅', '利好', '提振', '风险', '偏好', '期债', '低开', '低', '走', '国债', '期货', '全线', '收跌', '年期', '主力', '合约', '跌', '年期', '主力', '合约', '跌', '年期', '主力', '合约', '跌', '三大', '主力', '合约', '均', '创', '逾', '一年', '收盘', '新低', '行情', '解读', '公开市场', '方面', '央行', '开展', '亿元', '天期', '逆', '回购', '操作', '中标', '利率', '亿元', '逆', '回购', '到期', '因此', '当日', '净', '投放', '亿元', '资金面', '方面', '央行', '公开市场', '逆', '回购', '继续', '加力', '不过', '银行', '间', '市场', '周二', '资金', '供给', '趋于', '收敛', '七天', '回购', '加权', '利率', '进一步', '上行', '长期', '资金', '方面', '全国', '主要', '股份制', '银行', '一年期', '同业', '存单', '报价', '仍', '维持', '水平', '不过', '目前', '尚无', '成交量', '配合', '房地产', '行业', '再度', '迎来', '重磅', '利好', '证监会', '宣布', '涉房', '企业', '股权', '融资', '方面', '调整', '优化', '五项', '措施', '并', '自即日起', '施行', '随着', '证监会', '支持', '房企', '股权', '融资', '第三支', '箭', '正式', '发射', '信贷', '债券', '股权', '三个', '融资', '渠道', '三箭', '齐发', '合力', '推动', '房地产', '融资', '第三支', '箭', '快速', '落地', '将', '房地产', '行业', '注入', '大规模', '增量', '资金', '资产', '促进', '行业', '平稳', '健康', '发展', '短短', '天', '时间', '内', '政策', '端', '三箭',

In [251]:
#上述语句如果在数据量比较大的时候跑不出来，可以选择加一个n_jobs=1的参数，降低计算量，避免报错
lda_vis=gensimvis.prepare(lda_model,corpus,dictionary)

In [253]:
#显示可视化界面
pyLDAvis.display(lda_vis)